In [ ]:
! pip install numpy

import numpy as np
import math
np.__version__

In [ ]:
# Start by creating our game, which AlphaZero will play.
class TicTacToe:
    def __init__(self):
        self.row_count = 3
        self.column_count = 3
        # action size is the amount of possible actions (moves)
        self.action_size = self.row_count * self.column_count

    def get_initial_state(self):
        # new board = board full of zeros
        return np.zeros((self.row_count, self.column_count))

    def get_next_state(self, state, action: int, player):
        """
        state = board (2D array)
        action = 0-8 which cell to place move
        player = player
        """
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = player
        return state

    def get_valid_moves(self, state):
        # state.reshape() reshapes the matrix, in our case from 3x3 into something else
        # the -1 in state.reshape(-1) tells np to reshape automatically
        # in our case our 3x3 will become 1x9, the output is the new np matrix
        # looking like this: ([0, 0, 0, 0, 0, 0, 0, 0, 0])
        # by doing == 0 we get an output array like this:
        # np.array([True, True, True, True, True, True, True, True, True])
        # the last code, astype, tells np to convert the values into unsigned integers, making the array:
        # np.array([1, 1, 1, 1, 1, 1, 1, 1, 1])
        return (state.reshape(-1) == 0).astype(np.uint8)

    def check_win(self, state, action):
        if action == None:
            return False
            
        row = action // self.column_count
        column = action % self.column_count
        player = state[row, column]

        return (
            # Checks for rows, assume player is an integer
            np.sum(state[row, :]) == player * self.column_count
            or np.sum(state[:, column]) == player * self.row_count
            # Checks for diagonals, 
            or np.sum(np.diag(state)) == player * self.row_count
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count
        )

    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False

    def get_opponent(self, player):
        return -player

    def get_opponent_value(self, value):
        return -value

    def change_perspective(self, state, player):
        return state * player

In [ ]:
# normal MCTS

# Here, the node will never know of the player
# Instead, the rest of the code is programmed so that the state of the board
# will always be from the nodes perspective (i.e. flipped, or inverted might be a better word)
# For the node, the player is always player 1.
# Logic should be easier, and also valid for 1 player games
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken

        self.children = []
        self.expandable_moves = game.get_valid_moves(state)

        self.visit_count = 0
        self.value_sum = 0

    def is_fully_expanded(self):
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0

    def select(self):
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb

        return best_child

    def get_ucb(self, child):
        # What the child thinks of itself, not the parent,
        # Remember, the child is the opponent of the parent
        # That is why we take 1 - UCB
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count)/child.visit_count)

    def expand(self):
        # Choose one random expansion option
        action = np.random.choice(np.where(self.expandable_moves == 1)[0])
        self.expandable_moves[action] = 0

        child_state = self.state.copy()
        child_state = self.game.get_next_state(child_state, action, 1)
        child_state = self.game.change_perspective(child_state, player=-1)

        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child

    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)

        if is_terminal: return value
        # all other cases: rollout!
        rollout_state = self.state.copy()
        rollout_player = 1
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            if is_terminal:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                return value

            rollout_player = self.game.get_opponent(rollout_player)

    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1

        # flip player
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)
        

class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args

    def search(self, state):
        # define root 
        root = Node(self.game, self.args, state)
        for search in range(self.args['num_searches']):
            node = root
            # selection
            while node.is_fully_expanded():
                node = node.select()

            # The action taken here will be from the opponent,
            # Because of that, we take the negative value
            # That is, the value seen from our node
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)

            if not is_terminal:
                # expansion
                node = node.expand()
                # simulation
                value = node.simulate()
            # backpropagation
            node.backpropagate(value)
        
        # return visit_counts
        action_probs = np.zeros(self.game.action_size)
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        # Turn into probabilities, i.e. we want values in [0, 1].
        # In this MCTS, the node with the most amount of visits 
        # is deemed the most promising move
        action_probs /= np.sum(action_probs)
        return action_probs



In [ ]:
tictactoe = TicTacToe()
player = 1

args = {
    'C': 1.41,
    'num_searches': 1000
}

mcts = MCTS(tictactoe, args)

state = tictactoe.get_initial_state()

while True:
    print(state)

    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        print("valid moves:", [i for i in range(tictactoe.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}:"))
    
        if valid_moves[action] == 0:
            print("action not valid")
            continue
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        action = np.argmax(mcts_probs)
    
    state = tictactoe.get_next_state(state, action, player)

    value, is_terminal = tictactoe.get_value_and_terminated(state, action)

    if is_terminal:
        print(state)
        if value == 1:
            print(player, "won")
        else: 
            print("draw")
        break

    player = tictactoe.get_opponent(player)